In [1]:
import nbimporter
import numpy as np

In [2]:
import SolarPosision
from SolarPosision import SolarPosision

Importing Jupyter notebook from SolarPosision.ipynb


In [3]:
import Weather

Importing Jupyter notebook from Weather.ipynb


In [4]:
import Schedule

Importing Jupyter notebook from Schedule.ipynb
Importing Jupyter notebook from ACSet.ipynb
Importing Jupyter notebook from AnnualCal.ipynb
Importing Jupyter notebook from mdlLibrary.ipynb
Importing Jupyter notebook from Appl.ipynb
Importing Jupyter notebook from Light.ipynb
Importing Jupyter notebook from LocalVent.ipynb
Importing Jupyter notebook from Resi.ipynb


In [5]:
import WallMng
from WallMng import WallMng

Importing Jupyter notebook from WallMng.ipynb
Importing Jupyter notebook from Wall.ipynb


In [6]:
import ExsrfMng
from ExsrfMng import ExsrfMng

Importing Jupyter notebook from ExsrfMng.ipynb
Importing Jupyter notebook from Exsrf.ipynb


In [7]:
import WindowMng
from WindowMng import WindowMng

Importing Jupyter notebook from WindowMng.ipynb


In [8]:
import Sunbrk
from Sunbrk import SunbrkMng

Importing Jupyter notebook from Sunbrk.ipynb


# 室温・熱負荷を計算するクラス

## １）室内部位に関連するクラス

室内部位に関する情報を保持します。

- 外皮フラグ, 外皮の場合True
- 方位・隣室名, string
- 非定常フラグ, 非定常壁体の場合True
- 壁体・開口部名称, string
- 面積, m2
- ひさし名称, string
- 放射暖房吸収比率, －
- 人体に対する形態係数, －
- 床フラグ, 床の場合True（日射が多く当たる）

In [9]:
# 室内部位に関連するクラス
class Surface:
    
    # 初期化
    def __init__(self, skin, boundary, unsteady, name, area, sunbreak, flr, fot):
        self.__skin = skin            #外皮フラグ
        self.__boundary = boundary    #方位・隣室名
        
        # 外皮の場合は方位クラスを取得する
        if self.__skin == True :
            self.__objExsrf = ExsrfMng.ExsrfobjByName(self.__boundary)
        
        self.__unsteady = unsteady    #非定常フラグ
        self.__name = name            #壁体名称
        if '床' in self.__name :      #床フラグ
            self.__floor = True
        else:
            self.__floor = False
            
        self.__area = area            #面積
        self.__sunbreak = sunbreak    #ひさし名称
        self.__flr = flr              #放射暖房吸収比率
        self.__for = fot              #人体に対する形態係数
        
        # 壁体の初期化
        if self.__unsteady == True :
            self.__Row = WallMng.Row(self.__name)        #公比の取得
            self.__Nroot = WallMng.Nroot(self.__name)    #根の数
            self.__RFT0 = WallMng.RFT0(self.__name)      #貫流応答の初項
            self.__RFA0 = WallMng.RFA0(self.__name)      #吸熱応答の初項
            self.__RFT1 = WallMng.RFT1(self.__name)      #指数項別貫流応答の初項
            self.__RFA1 = WallMng.RFA1(self.__name)      #指数項別吸熱応答の初項
            self.__oldTsd_t[self.__nroot]                #貫流応答の表面温度の履歴
            self.__oldTsd_a[self.__nroot]                #吸熱応答の表面温度の履歴
            self.__hi = WallMng.hi(self.__name)          #室内側表面総合熱伝達率
            self.__hic = WallMng.hic(self.__name)        #室内側表面対流熱伝達率
            self.__hir = WallMng.hir(self.__name)        #室内側表面放射熱伝達率
            self.__ho = WallMng.ho(self.__name)          #室外側表面総合熱伝達率
            self.__as = WallMng.Solas(self.__name)       #室側側日射吸収率
            self.__Eo = WallMng.Eo(self.__name)          #室内側表面総合熱伝達率
            self.__oldTeo = 15.0                         #前時刻の室外側温度
            self.__olddqi = 0.0                          #前時刻の室内側表面熱流
        # 定常部位の初期化
        else:
            self.__tau = WindowMng.T(self.__name)        #日射透過率
            self.__B = WindowMng.B(self.__name)          #吸収日射取得率
            self.__Uso = WindowMng.Uso(self.__name)      #熱貫流率（表面熱伝達抵抗除く）
            self.__RFA0 = 1.0 / self.__Uso                 #吸熱応答係数の初項
            self.__RFT0 = 1.0                              #貫流応答係数の初項
            self.__hi = WindowMng.hi(self.__name)        #室内側表面総合熱伝達率
            self.__hic = WindowMng.hic(self.__name)      #室内側表面対流熱伝達率
            self.__hir = WindowMng.hir(self.__name)      #室内側表面放射熱伝達率
            self.__ho = WindowMng.ho(self.__name)        #室外側表面総合熱伝達率
            self.__U = 1.0 / (1.0 / self.__Uso + 1.0 / self.__hi)   #熱貫流率（表面熱伝達抵抗含む）
            self.__Eo = WindowMng.Eo(self.__name)        #室内側表面総合熱伝達率
            self.__sunbrkflg = len(self.__sunbreak) > 0  #庇がついているかのフラグ
    
    #床フラグの取得
    def Floor(self):
        return self.__floor
    
    #部位面積の取得
    def area(self):
        return self.__area
    
    #応答係数の初項の取得
    def RFA0(self):
        return self.__RFA0
    def RFT0(self):
        return self.__RFT0
    
    #指数項別応答係数
    def RFA1(self):
        return self.__RFA1
    def RFT1(self):
        return self.__RFT1
    
    #公比の取得
    def Row(self):
        if self.__unsteady == True:
            return self.__Row
        else:
            return 0
    
    #室内側表面総合熱伝達率
    def hi(self):
        return self.__hi
    
    #放射暖房吸収比率の取得
    def flr(self):
        return self.__flr
    
    #放射熱伝達率の取得
    def hir(self):
        return self.__hir
    
    #形態係数の設定
    def FF(self, i, FFd):
        self.__FF[i] = FFd
    #形態係数の取得
    def setFF(self, i):
        return self.__FF[i]

## ２）空間に関するクラス

In [10]:
# 空間に関する情報の保持
class Space:
    FsolFlr = 0.5                                 #床の日射吸収比率
    # 初期化
    def __init__(self,roomname,roomdiv,HeatCcap,HeatRcap,CoolCcap,Vol,Fnt,Vent,Inf,CrossVentRoom,RadHeat,Beta,NextVent,d):
        self.__roomname = roomname                #室用途（主たる居室、その他居室、非居室）
        self.__roomdiv = roomdiv                  #室名称（主寝室、子供室1、子供室2、和室）
        self.__AnnualLoadcC = 0.0                 #年間冷房熱負荷（対流成分）
        self.__AnnualLoadcH = 0.0                 #年間暖房熱負荷（対流成分）
        self.__AnnualLoadrC = 0.0                 #年間冷房熱負荷（放射成分）
        self.__AnnualLoadrH = 0.0                 #年間暖房熱負荷（放射成分）
        self.__Tr = 15.0                          #室温の初期化
        self.__oldTr = 15.0                       #前時刻の室温の初期化
        self.__kc = 0.5                           #人体表面の熱伝達率の対流成分比率
        self.__kr = 0.5                           #人体表面の熱伝達率の放射成分比率
        self.__HeatCcap = HeatCcap                #最大暖房能力（対流）
        self.__HeatRcap = HeatRcap                #最大暖房能力（放射）
        self.__CoolCcap = CoolCcap                #最大冷房能力（対流）
        self.__CoolRcap = CoolRcap                #最大冷房能力（放射）
        self.__Vol = Vol                          #室気積
        self.__Fnt = Fnt                          #家具熱容量
        self.__Hcap = self.__Vol * 1.2 * 1005. + self.__Vol * self.__Fnt * 1000.     # 室空気の熱容量（家具を含む）
        for i in range(1,3):
            self.__Vent[i] = Vent[i]              #計画換気量（1：冬期、2：中間期、3：夏期）
            self.__Inf[i] = Inf[i]                #すきま風量（1：冬期、2：中間期、3：夏期）
        self.__CrossVestRoom = CrossVentRoom      #通風対象室フラグ
        self.__RadHeat = RadHeat                  #放射暖房設置フラグ
        self.__Beta = Beta                        #放射暖房対流比率
        self.__oldNextRoom = []
        self.__NextVent = []
        for i in range(1,3):
            for j in range(0,len(NextVent)):
                self.__NextVent[i][j] = NextVent[i][j]    #室間換気量
                self.__oldNextRoom[j] = 15.0              #風上室の前時刻の温度
        self.__Nsurf = 0                           #部位の数
        self.__Surface = []
        for d_surface in d['Surface']:
            self.__Surface.append(Surface( d_surface['skin'], d_surface['boundary'], d_surface['unsteady'], \
                                          d_surface['name'], d_surface['area'], d_surface['sunbreak'], \
                                          d_surface['flr'], d_surface['fot']))
            self.__Nsurf += 1                         #定義部位数をインクリメント
        
        #面対面の形態係数の計算
        for i, surface in self.__Surface:
            self.__Atotal = 0.0
            
            #合計面積の計算
            for j, nxtsurface in self.__Surface:
                self.__Atotal += nxtsurface.area()
            
            #形態係数の計算（面積比）
            for j, nxtsurface in self.__Surface:
                self.surface.setFF[j] = nxtsurface.area() / self.__Atotal
                
        #放射収支計算のための行列準備
        #行列の準備と初期化
        #[AX]
        self.__matAXd = [ [ 0.0 for i in range(self.__Nsurf) ] for j in range(self.__Nsurf) ]
        #{FIA}
        self.__matFIA = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{CRX}
        self.__matCRX = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{CVL}
        self.__matCVL = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{FLB}
        self.__matFLB = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSC}
        self.__matWSC = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSR}
        #self.__matWSR = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSV}
        self.__matWSV = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        #{WSB}
        #self.__matWSB = [ [ 0.0 for i in range(1) ] for j in range(self.__Nsurf) ]
        
        for i, surface in self.__Surface:
            #matFIAの作成
            self.__matFIA[0][i] = surface.RFA0() * surface.hi()
            #FLB=φA0×flr×(1-Beta)
            self.__matFLB[0][i] = surface.RFA0() * surface.flr() * (1. - self.__Beta) / surface.area()
            
            #放射計算のマトリックス作成
            for j in self.__Nsurf:
                #対角要素
                if i == j:
                    self.__matAXd[i][j] = 1. + self.__matFIA[0][i] - surface.RFA0() * surface.hir() * surface.FF(j)
                #対角要素以外
                else:
                    self.__matAXd[i][j] = - surface.RFA0() * surface.hir() * surface.FF(j)
        
        #逆行列の計算
        self.__matAX = np.linalg.inv(self.__matAXd)
        #{WSR}の計算
        self.__matWSR = np.dor(self.__matAX, self.__matFIA)
        #{WSB}の計算
        self.__matWSB = np.dor(self.__matAX, self.__matFLB)
    
    #室温を返す
    def oldTr(self):
        return self.__oldTr
    
    #室温、熱負荷の計算
    def calcHload(self, SolarPosision, Schedule, Weather):
        #室間換気の風上室温をアップデート
        return 0

In [11]:
layers_test = [
    Layer( 'Plywood', 0.16, 720.0, 0.022 ),
    Layer( 'GW16K', 0.045, 13.0, 0.150 ),
]
wall_test = Wall( '1F_floor', 25.0, 0.90, 0.80, 1.70, 5.00, layers_test )
wall_test.AppendOutHeatTrans()
for i , layer in enumerate( wall_test.Layers() ):
    print( 'layer[' , i , '] Name:' , layer.name, ' R:',  layer.dblR(), '[(m2・K)/W]' )

NameError: name 'Layer' is not defined